In [4]:
import models
import pandas as pd
from sklearn.pipeline import make_pipeline
from preprocess import TypeConverter, KBinsDiscretizerWithNames, OneHotEncoderOnlyCategorical, CreateInteractions
from splitter import StratifiedRegressionSplit
from sklearn.metrics import r2_score
import numpy as np

In [9]:
variables = pd.read_excel('../data/covariates.xlsx')
data = pd.read_excel('../data/hp_ljubljana_new_with_rooms.xlsx')

variables = variables.to_dict(orient='records')

target = []
features = []
for feature in variables:
    feature['type'] = int if feature['type'] == 'int' else float if feature['type'] == 'float' else str
    if feature['group'] == 'target':
        target = feature
        continue
    else:
        features.append(feature)

X = data[[feature['name'] for feature in features]]
y = data[target['name']]

# PREPROCESS PIPLINE
preprocc_pip = make_pipeline(TypeConverter(variables=features), OneHotEncoderOnlyCategorical())
preprocc_pip.fit(X)
X = preprocc_pip.transform(X)

In [22]:
from sklearn.covariance import EmpiricalCovariance
from sklearn.feature_selection import r_regression, f_regression

corr = r_regression(X,y)
corr_f, p_values = f_regression(X, y)
corr_to_target = [{'feature': n, 'pearson_corr': c, 'f-value': f, 'p': p} for n, c, f, p in zip(X.columns, corr, corr_f,p_values)]



In [24]:
df = pd.DataFrame(data =corr_to_target)

In [25]:
df

,feature,pearson_corr,f-value,p
0,dist_airport,0.013343,1.012239,3.144099e-01
1,dist_highway_entr,0.076085,33.102088,9.200794e-09
2,dist_main_roads,0.065410,24.427234,7.938321e-07
3,dist_public_transport,0.044285,11.171337,8.360722e-04
4,dist_recreation,-0.069487,27.583231,1.559525e-07
5,dist_regional_roads,0.036322,7.509911,6.155204e-03
6,dist_river,-0.109816,69.395648,1.000003e-16
7,dist_schools,0.110069,69.719068,8.504531e-17
8,construct_age,-0.338840,737.367444,8.747235e-153
9,facade_age,-0.286861,509.760694,3.611249e-108


In [31]:
from sklearn.feature_selection import mutual_info_regression

mi = mutual_info_regression(X,y, n_neighbors=10)
mi_to_target = [{'feature': n, 'mi': m} for n, m in zip(X.columns, mi)]
df_mi = pd.DataFrame(data =mi_to_target)

In [32]:
df_mi

,feature,mi
0,dist_airport,5.152530e-02
1,dist_highway_entr,3.936258e-02
2,dist_main_roads,3.760590e-02
3,dist_public_transport,1.130815e-02
4,dist_recreation,3.893492e-02
5,dist_regional_roads,4.598221e-02
6,dist_river,2.289398e-02
7,dist_schools,2.792765e-02
8,construct_age,1.282795e-01
9,facade_age,8.496970e-02
